In [2]:
import pandas as pd
import numpy as np
import copy
import random

import datetime as datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from dython.nominal import associations
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.decomposition import PCA

from dython.nominal import associations
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.model_selection import train_test_split
from scipy import stats

from collections import OrderedDict 

from IPython.core.interactiveshell import InteractiveShell  # print all outputs
InteractiveShell.ast_node_interactivity = "all"

plt.style.use('fivethirtyeight') 
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['text.color'] = 'k'

%matplotlib inline

In [3]:
SEED = 1234
random.seed(SEED)

In [4]:
df_test = pd.read_csv("data/processed_test.csv")

In [5]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler

In [6]:
no_intervals = 5

In [7]:
# FIRST DISCRETE ONLY FOR WEATHER OF TOTAL DF (X)

#Weather_to_discretize are all the weather features

weather_to_discrete = []
weathers = ['temperature', 'rain_mm', 'humidity', 'wind_speed', 'pressure']
operators = ['mean', 'max', 'min']

#weather prev

for w in weathers:
    for o in operators:
        for D in ['7', '3']:
            weather_to_discrete.append(o + '_' + w + '_prev'+D+'d')
        
#weather future
for w in weathers:
    for o in operators:
        for D in ['7', '14']:
            weather_to_discrete.append(o + '_' + w + '_f_next'+D+'d')
            

In [8]:
discretizations = {
     
    'Ordinal-Equal-Width':KBinsDiscretizer(n_bins=no_intervals, encode='ordinal', strategy='uniform'),
    }

In [9]:
# df_to_discretize is the dataframe with features/columns I want to convert 
# original_df = the dataframe with all features
# Xd = discretized np-array

def discretized_to_df(Xd,df_to_discretize,original_df):
    scaled_features_df = pd.DataFrame(Xd, index=df_to_discretize.index, columns=df_to_discretize.columns)
    discretized_df = original_df.copy()
    for col in scaled_features_df.columns:
        if col in discretized_df.columns:
            discretized_df[col] = scaled_features_df[col]      
    return discretized_df 

#### Ordinal-Equal-Width is the best one ---> let's create the discretized dataframe

In [10]:
#Let's discretize also the test data
test_to_discretize = df_test[weather_to_discrete]    
test_d = discretizations['Ordinal-Equal-Width'].fit_transform(test_to_discretize) # Creates a np-array with discretized values
final_test = discretized_to_df(test_d, test_to_discretize,df_test) # creates a dataframe containing weather features with discretized value

### Save the dataframe

In [11]:
final_test.to_csv('data/processed_discrete_test_data.csv',index=False)